# RAG with llama index

Since llama index seems to have better frontend tooling with create-llama etc, here we attempt to redo a rag but with llama-index. We will reuse the embeddings that we created in [this notebook](./rag-llama-cloud-docs.ipynb).


In [1]:
%pip install llama-index chromadb --quiet
%pip install llama-index-vector-stores-chroma --quiet
%pip install llama-index-embeddings-ollama --quiet
%pip install llama-index-llms-ollama --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Here we turn debug logging on, so as to better understand what is going on.

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
import chromadb

chroma_client = chromadb.PersistentClient(path='./chroma')
col = chroma_client.get_collection('snappcloud')
print(col.count())

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
473


In [4]:
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

base_url="http://ollama-alibo-gpu-testing.apps.private.okd4.teh-2.snappcloud.io/"

ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:v1.5",
    base_url=base_url,
    ollama_additional_kwargs={"mirostat": 0},
)

llm = Ollama(model="llama3", base_url = base_url, request_timeout=120.0) #json_mode=True)

In [5]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = ollama_embedding

In [11]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext

vector_store = ChromaVectorStore(chroma_collection=col)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
#storage_context = StorageContext.from_defaults(vector_store=vector_store)
#index = VectorStoreIndex.from_documents(
#    documents, storage_context=storage_context, embed_model=embed_model
#)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store,embed_model=ollama_embedding,storage_context=storage_context)
query_engine = index.as_query_engine(llm=llm,response_mode="tree_summarize", verbose=True
)

In [12]:
resp = query_engine.query("What are the external IPs for snappcloud that i need to whitelist?")
print(resp)

INFO:httpx:HTTP Request: POST http://ollama-alibo-gpu-testing.apps.private.okd4.teh-2.snappcloud.io//api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://ollama-alibo-gpu-testing.apps.private.okd4.teh-2.snappcloud.io//api/chat "HTTP/1.1 200 OK"
According to the provided context information, the external IPs for SnappCloud that you need to whitelist are:

* 172.16.88.0/22 (for Afranet)
* 172.21.88.0/22 (for Asiatech)

These IPs can be accessed via VPN and are routable from the VPN.


In [8]:
result = query_engine.query("What address should I use for jaeger agent on snappcloud?")
print(result)

INFO:httpx:HTTP Request: POST http://ollama-alibo-gpu-testing.apps.private.okd4.teh-2.snappcloud.io//api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://ollama-alibo-gpu-testing.apps.private.okd4.teh-2.snappcloud.io//api/chat "HTTP/1.1 200 OK"
You need to use the following address for the Jaeger agent on SnappCloud: "jaeger - collector.service.consul: 14250".


In [9]:
resp = query_engine.query("I need to increase the quota for my project, how can I do this?")
print(resp)

INFO:httpx:HTTP Request: POST http://ollama-alibo-gpu-testing.apps.private.okd4.teh-2.snappcloud.io//api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://ollama-alibo-gpu-testing.apps.private.okd4.teh-2.snappcloud.io//api/chat "HTTP/1.1 200 OK"
To increase the quota for your project, you can run the command `oc edit quota` with the name of your project. This will allow the admin of each project to edit its project quota and update it to meet your needs. Please note that the quota of your projects is counted towards the team quota, so be careful not to exceed the team quota limit.
